# Mavlink Camera Server
> Mavlink experiments

[https://mavlink.io/en/mavgen_python/](https://mavlink.io/en/mavgen_python/)
[https://www.ardusub.com/developers/pymavlink.html](https://www.ardusub.com/developers/pymavlink.html)

https://mavlink.io/en/messages/common.html
https://mavlink.io/en/messages/common.html#MAV_TYPE
https://github.com/jackersson/gst-python-tutorials


In [ ]:
#| default_exp mavlink.cam_server

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
import UAV
from UAV.imports import *   # TODO why is this relative import on nbdev_export?
import cv2

import numpy as np
from imutils import resize

import time

from pathlib import Path
import time
from pymavlink import mavutil
import threading

import logging

import UAV.params as params

import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst, GObject, GLib

# gi.require_version('Gst', '1.0')

DEBUG  | matplotlib           | 19:55:42.426 |[__init__.py:305] MainThread | matplotlib data path: /home/jn/PycharmProjects/UAV/venv/lib/python3.10/site-packages/matplotlib/mpl-data
DEBUG  | matplotlib           | 19:55:42.430 |[__init__.py:305] MainThread | CONFIGDIR=/home/jn/.config/matplotlib
DEBUG  | matplotlib           | 19:55:42.432 |[__init__.py:1479] MainThread | interactive is False
DEBUG  | matplotlib           | 19:55:42.432 |[__init__.py:1480] MainThread | platform is linux
DEBUG  | matplotlib           | 19:55:42.503 |[__init__.py:305] MainThread | CACHEDIR=/home/jn/.cache/matplotlib
DEBUG  | matplotlib.font_manager | 19:55:42.506 |[font_manager.py:1543] MainThread | Using fontManager instance from /home/jn/.cache/matplotlib/fontlist-v330.json


In [ ]:
#| export
# logging.basicConfig(format='%(asctime)-8s,%(msecs)-3d %(levelname)5s [%(filename)10s:%(lineno)3d] %(message)s',
#                     datefmt='%H:%M:%S',
#                     level=params.LOGGING_LEVEL)  # Todo add this to params
# logger = logging.getLogger(params.LOGGING_NAME)

In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# from  UAV.gstreamer.valve import DefaultParams, GstStream
# from UAV.utils.display import show_image
# 
# gstcommand = DefaultParams().cameras["CAM-0"]["gst"]
# print(gstcommand)
# with  GstStream("CAM-0", gstcommand) as video:
#     time.sleep(2)
#     if video.frame_available():
#         image = video.frame()
#         ax = show_image(image, rgb2bgr=True)


In [ ]:
#| export
import logging, threading
class CameraServer:
    def __init__(self, connection_string, # "udpin:localhost:14550"
                 baudrate=57600, #baud rate of the serial port
                 camera_id=0, # camera id
                 gstpipes=[]): # list of gstreamer pipelines to control
        self._log = logging.getLogger("uav.{}".format(self.__class__.__name__))
        self.camera_id = camera_id
        # Create the connection
        self.master = mavutil.mavlink_connection(connection_string, baud=baudrate)
        self.gstpipes = gstpipes
        self._t_heartbeat = threading.Thread(target=self.send_heartbeat, daemon=True)
        self._t_heartbeat.start()
        self._t_mav_listen = threading.Thread(target=self.listen, daemon=True)
        self._t_mav_listen.start()
        self.image = None

    def __str__(self) -> str:
        return self.__class__.__name__

    def __repr__(self) -> str:
        return "<{}>".format(self)

    @property
    def log(self) -> logging.Logger:
        return self._log
    
    def send_heartbeat(self):
        """Send a heartbeat message to indicate the server is alive."""
        self._stop_threads = False
        while not self._stop_threads:
            self.master.mav.heartbeat_send(
                mavutil.mavlink.MAV_TYPE_CAMERA,  # type
                # mavutil.mavlink.MAV_TYPE_ONBOARD_CONTROLLER,
                mavutil.mavlink.MAV_AUTOPILOT_INVALID,  # autopilot
                0,  # base_mode
                0,  # custom_mode
                mavutil.mavlink.MAV_STATE_ACTIVE,  # system_status
                3  # MAVLink version
            )
            # print("Cam heartbeat_send")
            time.sleep(1)  # Send every second


    def listen(self):
        """Listen for MAVLink commands and trigger the camera when needed."""
        self._stop_threads = False
        self.log.info("Listening for MAVLink commands...")
        while not self._stop_threads:
            # Wait for a MAVLink message
            try:
                msg = self.master.recv_match(blocking=True, timeout=1)
            except Exception as e:
                self.log.error(e)
                continue
            # msg = self.master.recv_match(timeout=1)
            if not msg:
                continue
            # print (msg)

            # Check if it's a command to control the digital camera
            try:
                if msg.get_type() == 'COMMAND_LONG' and msg.command == mavutil.mavlink.MAV_CMD_DO_DIGICAM_CONTROL:
                    if msg.param2 == 1:  # check if the trigger capture parameter is set
                        self.trigger_camera(msg.param1)
                elif msg.get_type() == 'COMMAND_LONG' and msg.command == mavutil.mavlink.MAV_CMD_VIDEO_START_STREAMING:
                    if msg.param1 >= 1:
                        self.start_streaming(msg.param1)
            except Exception as e:
                self.log.error(e)
                continue
                
        self.log.info("Stopped Thread listening for MAVLink commands")

    def _start_stop_streaming(self, camera_id, dropstate=False):
        # start or stop video stream. 0 = all, 1 = primary camera, 2 secondary, etc.
        self.log.info(f"Camera {camera_id} dropstate: {dropstate}")
        return
        camera_id = int(camera_id-1)  # mavlink camera id starts at 1
        if camera_id >= len(self.gstpipes):
            logger.error(f"Camera {camera_id+1} not found")
        else:
            try:
                if camera_id == -1:
                    for pipe in self.gstpipes:
                        pipe.set_valve_state("myvalve", dropstate)
                else:
                    self.gstpipes[camera_id].set_valve_state("myvalve", dropstate)
            except exception as e:
                logger.error(e)
            
    def start_streaming(self, camera_id):
        # start video stream. 1 = primary camera, 2 secondary, etc.
        self._start_stop_streaming(camera_id, dropstate=False)

    def stop_streaming(self, camera_id):
        # stop video stream. 1 = primary camera, 2 secondary, etc.
        self._start_stop_streaming(camera_id, dropstate=True)    
        
    def trigger_camera(self,  camera_id):
        # Capture an image
        # if video.frame_available():
        #     self.image = video.frame()
            # ax = show_image(image)
            # self.pull_sample(camera_id)
            print(f"Camera triggered! Captured an image {'self.image.shape'}")

    def close(self):
        print("Closing camera server...")
        self.master.close()
        self.master.port.close()
        for pipe in self.gstpipes:
            pipe.shutdown()
        # self.gstpipes.close()
        self._stop_threads = True
        self._t_heartbeat.join()
        self._t_mav_listen.join()
        self.log.info(f"Camera {self.camera_id}  closed")
    
    
    def __enter__(self):
        """ Context manager entry point for with statement."""
        return self # This value is assigned to the variable after 'as' in the 'with' statement
    
    def __exit__(self, exc_type, exc_value, traceback):
        """Context manager exit point."""
        self.close()
        return False  # re-raise any exceptions
        


In [ ]:

with CameraServer("udpin:localhost:14550") as server:
    for i in range(100):
        time.sleep(0.1)
        # if server.image is not None:
        #     ax = show_image(server.image, rgb2bgr=True)
        #     server.image = None


INFO   | uav.CameraServer     | 19:55:42.804 |[262375905.py: 49] Thread-6 (listen) | Listening for MAVLink commands...


Closing camera server...


ERROR  | uav.CameraServer     | 19:55:53.346 |[262375905.py: 55] Thread-6 (listen) | [Errno 9] Bad file descriptor
INFO   | uav.CameraServer     | 19:55:53.347 |[262375905.py: 74] Thread-6 (listen) | Stopped Thread listening for MAVLink commands
INFO   | uav.CameraServer     | 19:55:53.845 |[262375905.py:119] MainThread | Camera 0  closed


In [ ]:
from UAV.mavlink.cam_server import CameraServer

from gstreamer import GstVidSrcValve 
import gstreamer.utils as utils

from UAV.utils.display import show_image
import time
DEFAULT_PIPELINE = utils.to_gst_string([
            'videotestsrc pattern=smpte is-live=true num-buffers=1000 ! tee name=t',
            't.',
            'queue leaky=2 ! valve name=myvalve drop=False ! video/x-raw,format=I420,width=640,height=480',
            'videoconvert',
            # 'x264enc tune=zerolatency noise-reduction=10000 bitrate=2048 speed-preset=superfast',
            'x264enc tune=zerolatency',
            'rtph264pay ! udpsink host=127.0.0.1 port=5000',
            't.',
            'queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=5/1,format=(string)BGR',
            'videoconvert ! appsink name=mysink emit-signals=true  sync=false async=false  max-buffers=2 drop=true ',
        ])

# print(DEFAULT_PIPELINE)
command = DEFAULT_PIPELINE
num_buffers = 80

with CameraServer("udpin:localhost:14550", gstpipes=[]) as server:
    # with GstVideoSourceValve(command, leaky=True) as pipeline:
    # with  GstStream("CAM-0", gstcommand) as video:
    pipeline = GstVidSrcValve(DEFAULT_PIPELINE)
    pipeline.startup()
    # server = CameraServer("udpin:localhost:14550", gstpipes=[pipeline])
    # server.listen()
    for i in range(50):
        time.sleep(0.1)
        buffer = pipeline.pop()
        # if video.frame_available():
        #     image = video.frame()
    pipeline.shutdown()
    # ax = show_image(buffer, rgb2bgr=True)

    # buffers = []
    # count = 0
    # dropstate = False
    # while len(buffers) < num_buffers:
    #     # time.sleep(0.1)
    #     count += 1
    #     if count % 20 == 0:
    #         # print(f'Count = : {count}')
    #         dropstate = not dropstate
    #         pipeline.set_valve_state("myvalve", dropstate)
    #     buffer = pipeline.pop()
    #     if buffer:
    #         buffers.append(buffer)
    # print('Got: {} buffers'.format(len(buffers)))

INFO   | uav.CameraServer     | 19:55:54.001 |[cam_server.py: 85] Thread-8 (listen) | Listening for MAVLink commands...
INFO   | pygst.GstVidSrcValve | 19:55:54.001 |[gst_tools.py:131] MainThread | GstVidSrcValve 
 gst-launch-1.0 videotestsrc pattern=smpte is-live=true num-buffers=1000 ! tee name=t t. ! queue leaky=2 ! valve name=myvalve drop=False ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc tune=zerolatency ! rtph264pay ! udpsink host=127.0.0.1 port=5000 t. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=5/1,format=(string)BGR ! videoconvert ! appsink name=mysink emit-signals=true  sync=false async=false  max-buffers=2 drop=true 
INFO   | pygst.GstVidSrcValve | 19:55:54.009 |[gst_tools.py:193] MainThread | Starting GstVidSrcValve
INFO   | pygst.GstVidSrcValve | 19:56:03.817 |[gst_tools.py:264] MainThread | GstVidSrcValve Shutdown requested ...
INFO   | pygst.GstVidSrcValve | 19:56:03.822 |[gst_tools.py:268] MainThread | Gst

Closing camera server...


INFO   | uav.CameraServer     | 19:56:04.041 |[cam_server.py:110] Thread-8 (listen) | Stopped Thread listening for MAVLink commands
INFO   | uav.CameraServer     | 19:56:04.043 |[cam_server.py:155] MainThread | Camera 0  closed


In [ ]:
from gstreamer import GstContext, GstPipeline, GstVidSrcValve 
import gstreamer.utils as utils
import time
DEFAULT_PIPELINE = utils.to_gst_string([
            'videotestsrc pattern=smpte is-live=true num-buffers=1000 ! tee name=t',
            't.',
            'queue leaky=2 ! valve name=myvalve drop=False ! video/x-raw,format=I420,width=640,height=480',
            'videoconvert',
            # 'x264enc tune=zerolatency noise-reduction=10000 bitrate=2048 speed-preset=superfast',
            'x264enc tune=zerolatency',
            'rtph264pay ! udpsink host=127.0.0.1 port=5000',
            't.',
            'queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=30/1,format=(string)BGR',
            'videoconvert ! appsink name=mysink emit-signals=true  sync=false async=false  max-buffers=2 drop=true ',
        ])

# print(DEFAULT_PIPELINE)
command = DEFAULT_PIPELINE
num_buffers = 80
with GstVideoSourceValve(command, leaky=True) as pipeline:
    buffers = []
    count = 0
    dropstate = False
    while len(buffers) < num_buffers:
        # time.sleep(0.1)
        count += 1
        if count % 20 == 0:
            # print(f'Count = : {count}')
            dropstate = not dropstate
            pipeline.set_valve_state("myvalve", dropstate)
        buffer = pipeline.pop()
        if buffer:
            buffers.append(buffer)
    print('Got: {} buffers'.format(len(buffers)))

NameError: name 'GstVideoSourceValve' is not defined

In [ ]:
from  UAV.gstreamer.valve import DefaultParams, GstStream
from UAV.utils.display import show_image

gstcommand = DefaultParams().cameras["CAM-0"]["gst"]
print(gstcommand)
with  GstStream("CAM-0", gstcommand) as video:

    server = CameraServer("udpin:localhost:14550", gstpipes=[video])
    # server.listen()
    for i in range(200):
        time.sleep(0.1)
        if video.frame_available():
            image = video.frame()
    
    ax = show_image(image, rgb2bgr=True)
            
    # server.listen()

### Mavlink Camera

In [ ]:
from  UAV.gstreamer.valve import DefaultParams, GstStream
from UAV.utils.display import show_image

pipes = []
for cam in list(DefaultParams.cameras.keys())[:2]:
    gstcommand = DefaultParams().cameras[cam]["gst"]
    print(gstcommand)
    pipes.append(GstStream(cam, gstcommand))

    
for pipe in pipes:
    pipe.close()
    
# gstcommand = DefaultParams().cameras["CAM-0"]["gst"]
# print(gstcommand)
# with  GstStream("CAM-0", gstcommand) as video:
# 
#     server = CameraServer("udpin:localhost:14550", gstpipes=[video])
#     # server.listen()
#     for i in range(200):
#         time.sleep(0.1)
#         if video.frame_available():
#             image = video.frame()
#     
#     ax = show_image(image, rgb2bgr=True)

In [ ]:
test_eq(1,1)

In [ ]:
server = CameraServer("udpin:localhost:14550")
try:
    server.listen()
except KeyboardInterrupt:
    server.close()

In [ ]:
assert False, "Stop here"

https://github.com/mavlink/MAVSDK/issues/1803

So I managed to change OpenHD in this regard.
No idea why I had such a hard time wrapping my head around, but now it works the following:
OpenHD binds port 127.0.0.1:14551 and listens on 127.0.0.1:14550
AND
instead of using sendto() with a unbound port (which then in turn means the sender port can be anything) messages are sent with sendto() from the bound port (the same that is used for listening).

So messages from OpenHD to mavsdk go the following:
OpenHD (out) via 127:0:0:1:14551 sent to 127:0:0:0:1:14550

So when mavsdk receives the first message, the sender address::port is 127:0:0:1:14551 and mavsdk can send the messages back to 127:0:0:1:14551.

https://julianoes.com/
The ports are not symmetrical! QGC listens on local port 14550 and sends UDP packets back to wherever messages came from.



In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev_export()